* Introduction

In building the fastq archives I missed copying the L2 files and forgot to uploaded.

Them when I built them themerge script ran on systems without access to the flowcell archive on packrat and so I ended up with empty files.

I need to resubmit them to the DCC

In [ ]:
import pandas
import os
import paramiko
import json
from glob import glob

In [ ]:
from curation_common import *

In [ ]:
from htsworkflow.util import hashfile

In [ ]:
encode = ENCODED('www.encodeproject.org')
encode.load_netrc()

In [ ]:
all_files = {
'16930/16930_HLGVVBCXX_c116_l2.fastq.gz': "/files/ENCFF262CIY/",
'16929/16929_HLGVVBCXX_c116_l2.fastq.gz': "/files/ENCFF500NCE/",
'16954/16954_HLGG7BCXX_c116_l2.fastq.gz': "/files/ENCFF216NUY/",
'16923/16923_HLHGNBCXX_c116_l2.fastq.gz': "/files/ENCFF700OLU/",
'16924/16924_HLHGNBCXX_c116_l2.fastq.gz': "/files/ENCFF853SOX/",
'16937/16937_HLHJMBCXX_c116_l2.fastq.gz': "/files/ENCFF078SPI/",
'16940/16940_HLHJMBCXX_c116_l2.fastq.gz': "/files/ENCFF509HFH/",
'16939/16939_HLHJMBCXX_c116_l2.fastq.gz': "/files/ENCFF887HOU/",
'16933/16933_HLGVVBCXX_c116_l2.fastq.gz': "/files/ENCFF551UCM/",
'16944/16944_HLGH2BCXX_c116_l2.fastq.gz': "/files/ENCFF065YUF/",
'16957/16957_HLGG7BCXX_c116_l2.fastq.gz': "/files/ENCFF198SLH/",
'16950/16950_HLGH2BCXX_c116_l2.fastq.gz': "/files/ENCFF306XSY/",
'16927/16927_HLHGNBCXX_c116_l2.fastq.gz': "/files/ENCFF160LUK/",
'16949/16949_HLGH2BCXX_c116_l2.fastq.gz': "/files/ENCFF974EKR/",
'16943/16943_HLGH2BCXX_c116_l2.fastq.gz': "/files/ENCFF737TKD/",
'16934/16934_HLGVVBCXX_c116_l2.fastq.gz': "/files/ENCFF854WTE/",
'16941/16941_HLHJMBCXX_c116_l2.fastq.gz': "/files/ENCFF267VJG/",
'16936/16936_HLHJMBCXX_c116_l2.fastq.gz': "/files/ENCFF555MUK/",
'16925/16925_HLHGNBCXX_c116_l2.fastq.gz': "/files/ENCFF476UXE/",
'16922/16922_HLHGNBCXX_c116_l2.fastq.gz': "/files/ENCFF294JRP/",
'16955/16955_HLGG7BCXX_c116_l2.fastq.gz': "/files/ENCFF106HJE/",
'16928/16928_HLHGNBCXX_c116_l2.fastq.gz': "/files/ENCFF063CEM/",
'16931/16931_HLGVVBCXX_c116_l2.fastq.gz': "/files/ENCFF237SXT/",
'16935/16935_HLGVVBCXX_c116_l2.fastq.gz': "/files/ENCFF155LJD/",
'16942/16942_HLHJMBCXX_c116_l2.fastq.gz': "/files/ENCFF277ETM/",
'16948/16948_HLGH2BCXX_c116_l2.fastq.gz': "/files/ENCFF824LLV/",
'16926/16926_HLHGNBCXX_c116_l2.fastq.gz': "/files/ENCFF876NSY/",
'16951/16951_HLGG7BCXX_c116_l2.fastq.gz': "/files/ENCFF301BEB/",
'16956/16956_HLGG7BCXX_c116_l2.fastq.gz': "/files/ENCFF987HYN/",
'16945/16945_HLGH2BCXX_c116_l2.fastq.gz': "/files/ENCFF216DGZ/",
'16932/16932_HLGVVBCXX_c116_l2.fastq.gz': "/files/ENCFF771GDS/",
'16938/16938_HLHJMBCXX_c116_l2.fastq.gz': "/files/ENCFF286QHA/",        
}

In [ ]:
submission_root = '/woldlab/castor/home/diane/proj/submission/encode-201604'
pongo = paramiko.SSHClient()
pongo.set_missing_host_key_policy(paramiko.WarningPolicy())
pongo.connect('pongo.cacr.caltech.edu', username='diane')
sftp = pongo.open_sftp()
sftp.chdir(submission_root)

In [ ]:
def sftp_exists(sftp, filename):
    try:
        stat = sftp.stat(filename)
    except FileNotFoundError as e:
        return False
    return True

In [ ]:
def reupload_credentials(accession):
    response = encode.post_json(accession + 'upload', {})
    if response.get('status') == 'success':
        metadata = response['@graph'][0]
        print('creds =', metadata['upload_credentials'])
        print("run_aws_cp('{}', creds)".format(metadata['submitted_file_name']))
        return metadata


In [ ]:
def save_success(sftp, metadata):
    upload_filename = metadata.get('submitted_file_name') + '.upload'
    if sftp_exists(sftp, upload_filename):
        print('weird {} exists'.format(upload_filename))
    else:
        upload_file = sftp.open(upload_filename, 'wa')
        upload_file.write(json.dumps(metadata, indent=4, sort_keys=True))
        upload_file.close()

In [ ]:
bad_files = {}
for filename in all_files:
    upload_filename = filename + '.upload'
    if sftp_exists(sftp, upload_filename):
        print('{} was uploaded'.format(filename))
    else:
        bad_files[filename] = all_files[filename]
        print('{} was NOT uploaded'.format(filename))


In [ ]:
for filename in bad_files:
    try:
        stat = sftp.stat(filename)
        if stat.st_size < 100:
            print('{} too small {}'.format(filename, stat.st_size))
    except FileNotFoundError as e:
        print('{} not found'.format(filename))


In [ ]:
bad_files

Uncomment the following to refresh upload credentials

In [ ]:
#metadata = reupload_credentials()

In [ ]:
#metadata['submitted_file_name']

In [ ]:
#save_success(sftp, metadata)

Need to fix metadata

In [ ]:
model = get_model(
    'encode', 
    directory=os.path.expanduser('~/proj/submission/encode-201604/'), 
    use_contexts=False)

In [ ]:
%%sparql -m model
select ?s ?md5
where {
    ?s <http://jumpgate.caltech.edu/wiki/UcscDaf#md5sum> ?md5 ;
       
}


In [ ]:
upload_files = glob(os.path.expanduser('~/proj/submission/encode-201604/*/*_l2.fastq.gz.upload'))

for filename in upload_files:
    with open(filename, 'rt') as instream:
        body = json.load(instream)
        if '@graph' in body:
            data = body['@graph'][0]
        else:
            data = body
        for key in ['@id', 'file_size', 'md5sum', 'submitted_file_name']:
            print(key, data[key])
        print()

In [ ]:

curdir = os.getcwd()
os.chdir(os.path.expanduser('~/proj/submission/encode-201604/'))

for filename in upload_files:
    with open(filename, 'rt') as instream:
        body = json.load(instream)
        if '@graph' in body:
            data = body['@graph'][0]
        else:
            data = body
        
        remote_data = encode.get_json(data['@id'])
        submitted_file_name = data['submitted_file_name']
        md5sum = hashfile.make_md5sum(submitted_file_name)
        stat = os.stat(submitted_file_name)
        payload = {'file_size': stat.st_size, 'md5sum': md5sum}
        do_update = False
        
        for key in ['md5sum', 'file_size']:
            if remote_data[key] != payload[key]:
                print("Upd: {} {} -> {}".format(data['@id'], payload[key], remote_data[key]))
                do_update = True

        if do_update:
            result = encode.patch_json(data['@id'], payload)
            result_data = result['@graph'][0]
            print('Res:', result_data['@id'], result_data['md5sum'], result_data['file_size'])
            do_update = False
os.chdir(curdir)

In [ ]:
%debug

In [ ]:
data = encode.get_json('/files/ENCFF063CEM/')

In [ ]:
encode.patch_json?

In [ ]:
body

In [ ]:
testmodel = get_model(use_contexts=False)
load_jsonld_into_model(testmodel, body['@graph'][0])

In [ ]:
%%sparql -m testmodel
select ?s ?p ?o
where {
    ?s ?p ?o .
}

In [ ]:
len(testmodel)